In [1]:
import nltk

# from nltk.parse.stanford import StanfordDependencyParser
# path_to_jar = '/Development/Projects/Magnifis/3rd_party/NLU/stanford-corenlp-full-2013/stanford-corenlp-3.2.0.jar'
# path_to_models_jar ='/Development/Projects/Magnifis/3rd_party/NLU/stanford-corenlp-full-2013/stanford-corenlp-3.2.0-models.jar'
# dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)

# result = dependency_parser.raw_parse('I shot an elephant in my sleep')
# dep = result.next()
# list(dep.triples())


In [2]:
import os
import io
raw_corpus = ''
dir_name = './similes_train/'   
fList=os.listdir(dir_name)
fList1 = [os.path.join(dir_name, f) for f in fList if os.path.isfile(os.path.join(dir_name, f))] # Create a list of file names with full path, eliminating subdirectory entries


In [3]:
import codecs

#max_files = 1000 #remove to get the entire corpus
raw_corpus = ''
for file in fList1: #[0:max_files] 
    with codecs.open(file, 'r', 'utf-8') as f:
    #with open(file, encoding="utf8") as f:
        raw_corpus += ''.join(f.read())   


In [4]:
import re
corpus = re.sub(r"(\n|\r)+""|(@\w+)+", ' ', raw_corpus) #remove backslashes and words starting with @
#corpus = re.sub(r"(as soon)+" "|(as well)+" "|(as if)+" "|(as quickly as possible)+" "|(as long)" "|(as usual)+" "|(such as)+" "|(as of yet)+" "|(as much)+" "|(as many)+" "|(like that)+" "|(like this)+" "|(like you)+" "|(like me)+" "|(like him)+" "|(like us)+" "|(like her)+" "|(anybody like)+" "|(anyone like)+", "", corpus)
regex_filter = r"(as soon)|(as well)|(as if)|(as \w+ as possible)|(as before)|\
(as long)|(as usual)|(as ever)|\
(such as)|(as of yet)|(as much)|(as many)|\
(like that)|(like this)|(like you)|(like me)|(like him)|(like us)|(like her)|\
(like everything else)|(like everyone else)|(anybody like)|(anyone like)"


In [5]:
re.search(regex_filter, u'as long as') 

<_sre.SRE_Match object; span=(0, 7), match='as long'>

**the algorithm:**
#take a big corpus with trite similes (The Daily Mail, http://cs.nyu.edu/~kcho/DMQA/, or a pulp fiction book), prepare clean sentences, traverce throug the corpus sen by sen, POS-tagging each and looking for sentences containing prepositions "like" and "as" (tag = "IN") and exclude "as soon as, as well as, as usual, such as, as of yet, as much, like that, like this.." ((alternatively, use dependency parser to accurately cut out a phrase. but it's a pain and may be an overkill)) Add these sentences to a target corpus. Cut out a simile candidate out of each sentence; optionally: replace "likes" and "ases" with a "comparator". 
#approach 1:
#Use fuzzywazzy to do fuzzy matching of the simile candidates across the corpus. Find those that at least 98% similar and appear multiple times (over 10) across the corpus - those are thrite similes (or common grammatical constuctions containing 'like' or 'as' that we missed during cleaning). Build a corpus of trite similes. With a testing set, repeat all steps up to fuzzywazzy. Then, instead of fuzzy matching candidates across the testinf set, fuzzy match them with the trite similes corpus. Highlight (tag) if a match is found.  
#approach 2: 
#Sort words in each set alphabetically. Then build an n-gram counter (may be plot a histogram) - a dictionary with an n-gram as a key and how many times is appears in the corpus as a value. In a new text, repeat all steps up to the last one and then find new n-grams in the dictionary. If the new n-grams are among the most frequently met n-grams in the corpus, these n-grams constitute trite similes. Then use them as a trite similes corpus and compare to the testing set as described in the approach 1. 


In [6]:
def tokenize_text(corpus, regex_filter):
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sents = sent_tokenizer.tokenize(corpus) # Split text into sentences 
    if regex_filter:
        raw_sents = [sent for sent in raw_sents if not re.search(regex_filter, sent)]
    return [nltk.word_tokenize(word) for word in raw_sents]

In [7]:
sentences = tokenize_text(corpus, regex_filter)

In [8]:
sentences[10]

['He',
 'had',
 'just',
 'entered',
 ',',
 'wearing',
 'an',
 'embroidered',
 'court',
 'uniform',
 ',',
 'knee',
 'breeches',
 ',',
 'and',
 'shoes',
 ',',
 'and',
 'had',
 'stars',
 'on',
 'his',
 'breast',
 'and',
 'a',
 'serene',
 'expression',
 'on',
 'his',
 'flat',
 'face',
 '.']

In [9]:
comparisons = []
for sent in sentences:
    if not 'like' in sent and not 'as' in sent: 
        continue 
    # exlude a single 'as', leaving in only '...as ... as...'
    if not 'like' in sent and len([word for word in sent if word=='as']) == 1: 
        continue
    pos_tagged = nltk.pos_tag(sent)
    for pair in pos_tagged:
        if pair[1] == 'IN' and (pair[0] == 'like' or pair[0] == 'as'):
            comparisons.append(pos_tagged)


In [10]:
comparisons[0:10]

[[('Can', 'MD'),
  ('one', 'CD'),
  ('be', 'VB'),
  ('calm', 'VBN'),
  ('in', 'IN'),
  ('times', 'NNS'),
  ('like', 'IN'),
  ('these', 'DT'),
  ('if', 'IN'),
  ('one', 'CD'),
  ('has', 'VBZ'),
  ('any', 'DT'),
  ('feeling', 'NN'),
  ('?', '.'),
  ('”', 'NN'),
  ('said', 'VBD'),
  ('Anna', 'NNP'),
  ('Pávlovna', 'NNP'),
  ('.', '.')],
 [('I', 'PRP'),
  ('confess', 'VBP'),
  ('all', 'PDT'),
  ('these', 'DT'),
  ('festivities', 'NNS'),
  ('and', 'CC'),
  ('fireworks', 'NNS'),
  ('are', 'VBP'),
  ('becoming', 'VBG'),
  ('wearisome.”', 'NNS'),
  ('“If', 'IN'),
  ('they', 'PRP'),
  ('had', 'VBD'),
  ('known', 'VBN'),
  ('that', 'IN'),
  ('you', 'PRP'),
  ('wished', 'VBD'),
  ('it', 'PRP'),
  (',', ','),
  ('the', 'DT'),
  ('entertainment', 'NN'),
  ('would', 'MD'),
  ('have', 'VB'),
  ('been', 'VBN'),
  ('put', 'VBN'),
  ('off', 'RP'),
  (',', ','),
  ('”', 'NNP'),
  ('said', 'VBD'),
  ('the', 'DT'),
  ('prince', 'NN'),
  (',', ','),
  ('who', 'WP'),
  (',', ','),
  ('like', 'IN'),
  ('a', '

In [11]:
similes_candidates = []
key_pos_tags = set(['NN', 'NNS', 'NNP', 'VB', 'VBN', 'VBD', 'VBG']) # noun or verb

for tagged_sent in comparisons:
    start_index = -1
    words_after = -1
    sent = [pair[0] for pair in tagged_sent]
    pos_tags = [pair[1] for pair in tagged_sent]
    if 'like' in sent:
        start_index = sent.index('like')
        #two_words_before_like = max(0, index_of_like - 4)
        words_after = min(len(sent), start_index + 6)
    elif 'as' in sent:
        start_index = sent.index('as')
        words_after = min(len(sent), start_index + 8)
        
    if start_index >= 0 and words_after > 0:
        try: 
            index_of_punkt = max(sent.index(','), 0)
            if index_of_punkt > start_index: 
                words_after = min(words_after, index_of_punkt)
        except ValueError: # just ignore
            pass
    
    if not(not key_pos_tags.intersection(set(pos_tags[start_index:words_after]))): # make sure at least one key pos tag is present
        similes_candidates.append(sent[start_index:words_after])


In [12]:
similes_candidates[:100]

[['like', 'a', 'wound-up', 'clock', ',', 'by'],
 ['like', 'an', 'actor', 'repeating', 'a', 'stale'],
 ['like', 'it', ',', 'she', 'became', 'enthusiastic'],
 ['like', 'a', 'child', 'in', 'a', 'toyshop'],
 ['as', 'a', 'specially', 'choice', 'delicacy', 'a', 'piece', 'of'],
 ['as', 'a', 'specially', 'choice', 'delicacy', 'a', 'piece', 'of'],
 ['like', 'a', 'well-garnished', 'joint', 'of', 'roast'],
 ['like', 'his', 'sister’s'],
 ['like', 'Russia—barbaric', 'as', 'she', 'is', 'said'],
 ['like', 'Russia—barbaric', 'as', 'she', 'is', 'said'],
 ['like', 'a', 'Frenchman', ',', '“has', 'been'],
 ['like', 'a', 'Frenchman', ',', '“has', 'been'],
 ['like', 'qualms', 'of', 'conscience', '.'],
 ['like', 'the', 'vicomte', 'and', 'was', 'aiming'],
 ['like', 'the', 'conduct', 'of', 'a', 'great'],
 ['as', 'a', 'private', 'person', ',', 'as', 'a', 'general'],
 ['as', 'a', 'private', 'person', ',', 'as', 'a', 'general'],
 ['as', 'a', 'private', 'person', ',', 'as', 'a', 'general'],
 ['like', 'one', 'quite

In [13]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(['a', 'an', 'and', 'or', 'the', \
                  'his', 'her', 'my', 'your', 'their', 'our', \
                  'i', 'you', 'he', 'she', 'it', 'they', 'who', 'that', 'whose', \
                  'is', 'are', 'was', 'will', 'would', \
                  '.',',','-',':',';','!','?'])

def preprocess_words(wordlist): 
    wordset = set([])
    for word in wordlist: 
        word = word.lower()
        if word not in stop_words and len(word) > 1: 
            if word != 'as':
                word = lemmatizer.lemmatize(word)
            if word == 'like' or word == 'as': 
                word = '$cmpr'
            wordset.add(word)
    return wordset 
        

def init_corpus_2match(wordlists): 
    wordsets = []
    for words in wordlists: 
        if not words:
            continue
        wordset = preprocess_words(words)
        if not(not wordset):
            wordsets.append(wordset)
    return wordsets


''' Returns a list of matches for 'phrase' in 'wordsets' with 'min_similarity' 
'''
def fuzzy_match(words_in, corpus, min_similarity): 
    # TODO: must be optimized!!
    phraset = preprocess_words(words_in)
    #print ("Input phraseset is {}".format(phraset))
    nb_input = len(phraset)
    matches = []
    for wordset in corpus: 
        intersect = phraset.intersection(wordset)
        n = len(intersect)
        if n/min(nb_input, len(wordset)) >= min_similarity and not(n < 2 and next(iter(intersect))=='$cmpr'): 
            #print(wordset)
            matches.append(wordset)
    return matches
        





In [14]:
candidates = similes_candidates
corpus_2match = init_corpus_2match(candidates)
covered = set([])
count_dict = {}
for cand in candidates:
    if not cand: 
        continue
    phrase = ' '.join(cand)
    if phrase in covered:
        continue
    covered.add(phrase)
    result = fuzzy_match(cand, corpus_2match, 0.75)
    #print("result is {}".format(result))
    if result:
        count_dict[phrase] = len(result)


In [15]:
import operator
sorted_counts = sorted(count_dict.items(), key=operator.itemgetter(1))
sorted_counts.reverse()
sorted_counts

[('like a man .', 39),
 ('like a child', 26),
 ('as a child', 26),
 ('like a child .', 26),
 ('like it ? ” said a', 13),
 ('like it to have been ,', 11),
 ('like the prince', 11),
 ('like the others', 11),
 ('like a child in a toyshop', 11),
 ('like the others , and like', 11),
 ('like children in an embarrassing situation', 11),
 ('as Natásha and I used to as children', 11),
 ('like a child when the doctor', 10),
 ('like a son .', 10),
 ('as necessary for the movement of the peoples', 10),
 ('like a suffering child and to', 10),
 ('like the first movement of the', 9),
 ('like a girl ( it was', 9),
 ('like to be an adjutant and', 9),
 ('as interesting as he had been to his', 9),
 ('like a child taken out for', 9),
 ('like to be an hussar .', 9),
 ('as those that had been addressed to her', 9),
 ('like the movement of the wheels', 9),
 ('like children of my own ,', 9),
 ('like it and did not want', 9),
 ('like a dream in which a', 9),
 ('like a child till tears came', 8),
 ('like a chil

In [16]:
from sklearn.externals import joblib
min_freq = 5

top_results_corpus = init_corpus_2match([item[0].split(' ') for item in count_dict.items() if item[1] >= min_freq])

# save 
joblib.dump(count_dict, "top_similes_corpus.v0.pkl")

ImportError: No module named 'sklearn'

In [ ]:
## simile classification code ##

# similes_corpus = joblib.load("top_similes_corpus.v0.pkl")
# matches = fuzzy_match(new_sent_words, similes_corpus, 0.75)
# if len(matches) >= min_freq
#     print("'{}' is a trite simile".format(new_sent_words))
# else 
#     print("'{}' is NOT a trite simile".format(new_sent_words))



## Backup code 

In [ ]:
# import fuzzywuzzy
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process


In [ ]:
# choices = []
# for each in similes_candidates:
#     choices.append(" ".join(each))


In [ ]:
# count_dict = {}

# for string in set(choices):
#     result = process.extract(string, choices, limit=1000) #default limit = 5
#     num_matches = 0
#     for each in result:
#         if each[1] > 98:
#             num_matches +=1
#     count_dict[string] = num_matches


In [ ]:
# write 
# from sklearn.externals import joblib
# joblib.dump(count_dict, "count_dict_output.pkl")

In [ ]:
# count_dict = sorted(count_dict.items(), key=operator.itemgetter(1))
# count_dict.reverse()


In [ ]:
#read 
#count_dict_fromfile = joblib.load("count_dict_output.pkl")